# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846749286126                   -0.70    4.5         
  2   -7.852302873830       -2.26       -1.53    1.0   27.4ms
  3   -7.852615053652       -3.51       -2.55    1.5   29.0ms
  4   -7.852646014605       -4.51       -2.89    2.8   36.0ms
  5   -7.852646525876       -6.29       -3.20    1.0   25.9ms
  6   -7.852646680080       -6.81       -4.06    1.0   25.9ms
  7   -7.852646686382       -8.20       -5.19    1.8   31.4ms
  8   -7.852646686724       -9.47       -5.48    2.0   32.8ms
  9   -7.852646686730      -11.27       -6.45    1.2   27.5ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846828883477                   -0.70           4.5         
  2   -7.852525730550       -2.24       -1.64   0.80    2.0    275ms
  3   -7.852636221103       -3.96       -2.73   0.80    1.0   24.6ms
  4   -7.852646481247       -4.99       -3.26   0.80    2.2   95.1ms
  5   -7.852646671182       -6.72       -4.08   0.80    1.0   23.1ms
  6   -7.852646686387       -7.82       -4.75   0.80    1.8   28.8ms
  7   -7.852646686725       -9.47       -5.61   0.80    1.8   29.0ms
  8   -7.852646686730      -11.35       -6.97   0.80    1.8   27.7ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.399269e+01     3.744021e+00
 * time: 0.06218695640563965
     1     1.077707e+00     2.016979e+00
 * time: 0.3026590347290039
     2    -1.416457e+00     2.446651e+00
 * time: 0.3280520439147949
     3    -3.597063e+00     1.906342e+00
 * time: 0.36466503143310547
     4    -4.881931e+00     1.689934e+00
 * time: 0.40033411979675293
     5    -6.533894e+00     1.007432e+00
 * time: 0.4365570545196533
     6    -7.161635e+00     5.676193e-01
 * time: 0.47236108779907227
     7    -7.442690e+00     2.117662e-01
 * time: 0.49747610092163086
     8    -7.588319e+00     1.910843e-01
 * time: 0.5224349498748779
     9    -7.680847e+00     1.356620e-01
 * time: 0.5476410388946533
    10    -7.736923e+00     1.241354e-01
 * time: 0.5722270011901855
    11    -7.785306e+00     1.290501e-01
 * time: 0.5972621440887451
    12    -7.814841e+00     9.404521e-02
 * time: 0.6220569610595703
    13    -7.839426e+00     5.003009e-02
 * time: 0.64709

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846891217899                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645931991                   -1.64         
  2   -7.852646686730       -6.12       -3.71    2.10s
  3   -7.852646686730      -13.38       -7.26    169ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.8278987417627325e-7
|ρ_newton - ρ_scfv| = 2.0555561696624974e-7
|ρ_newton - ρ_dm|   = 1.790915459234508e-9
